In [ ]:
!pip install -r Requirements.txt

In [72]:
#Imports
import pandas as pd
import numpy as np
import datetime as DT
from sklearn.preprocessing import LabelEncoder


import warnings
warnings.filterwarnings('ignore')

In [2]:
#user dataset given by the company
user_data= pd.read_parquet('/work/project_maternity/data/users/data_users.parquet')

In [3]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105914 entries, 0 to 105913
Data columns (total 20 columns):
 #   Column                            Non-Null Count   Dtype              
---  ------                            --------------   -----              
 0   Userinfo_Key                      105914 non-null  object             
 1   Userinfo_BK                       105914 non-null  object             
 2   Country_Key                       105914 non-null  object             
 3   UserinfoAppId                     105914 non-null  object             
 4   RN                                105914 non-null  float64            
 5   OnlyUser                          105914 non-null  object             
 6   NumberOfUsers                     105914 non-null  object             
 7   UserinfoCountry                   105914 non-null  object             
 8   UserinfoState                     105914 non-null  object             
 9   UserinfoDistrict                  105914 non-nul

In [4]:
#converting object to numeric
user_data['Userinfo_Key']=pd.to_numeric(user_data['Userinfo_Key'])

# Imputing locations from activity dataset

In [5]:
# location dataset getting from the lat and log for each user 
data_location= pd.read_csv('Found_users_location_data.csv')

In [6]:
data_location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1607 entries, 0 to 1606
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Userinfo_Key  1607 non-null   int64 
 1   UserDistrict  1607 non-null   object
 2   UserCountry   1607 non-null   object
 3   UserState     1607 non-null   object
dtypes: int64(1), object(3)
memory usage: 50.3+ KB


In [7]:
# checking every user is unique inside the location datset
len(data_location['Userinfo_Key'].unique())

1607

In [8]:
# Making sure user in data_location dataset are all inside the user_data
data_location['Userinfo_Key'].isin(user_data['Userinfo_Key']).sum()

1607

In [9]:
data_location.head()

,Userinfo_Key,UserDistrict,UserCountry,UserState
0,474737,Nabarangapur,India,Odisha
1,474986,Jammu District,India,Jammu and Kashmir
2,476738,Faridabad,India,Haryana
3,481106,Kanpur Nagar,India,Uttar Pradesh
4,481511,Pauri Garhwal,India,Uttarakhand


In [10]:
user_data[user_data['Userinfo_Key']==474737]

,Userinfo_Key,Userinfo_BK,Country_Key,UserinfoAppId,RN,OnlyUser,NumberOfUsers,UserinfoCountry,UserinfoState,UserinfoDistrict,HealthcareWorker,Profession,Experiance,WorkPlace,SafeDeliveryApp,UserinfoNumberOfDelivery,UserinfoIsMyLearning,UserinfoUserIsActive,UserInfoFirstMyLearningEventDate,UserInfoFirstUseDate
219,474737,157dea60-696f-11e8-8d1f-35d06538fae1,487380,157dea60-696f-11e8-8d1f-35d06538fae1,4.0,Unknown,Unknown,India,Unknown,Unknown,Yes,Unknown,Unknown,Primary health facility,Unknown,50+,1.0,0.0,2018-06-09 00:00:00+00:00,2018-06-06 00:00:00+00:00


In [11]:
data_location[data_location['Userinfo_Key']==474737]

,Userinfo_Key,UserDistrict,UserCountry,UserState
0,474737,Nabarangapur,India,Odisha


In [12]:
#unknown values before 
print(user_data[user_data['UserinfoDistrict']== 'Unknown'].shape)
print(user_data[user_data['UserinfoState']== 'Unknown'].shape)

(25225, 20)
(13147, 20)


In [13]:
#looking into data_location dataset (where we have values for state and district), 
#imputing the values of userinfoDistrict and UserinfoState based on Userinfo_Key

for index, row in data_location.iterrows():
    row_district = row['UserDistrict']
    row_state = row['UserState']
    row_user = row['Userinfo_Key']
    
    user_data.loc[user_data['Userinfo_Key']==row_user, 'UserinfoDistrict'] = row_district
    user_data.loc[user_data['Userinfo_Key']==row_user, 'UserinfoState'] = row_state

In [14]:
#unknown values after
print(user_data[user_data['UserinfoDistrict']== 'Unknown'].shape)
print(user_data[user_data['UserinfoState']== 'Unknown'].shape)

(23618, 20)
(11924, 20)


# looking into activity dataset

In [15]:
data_activity = pd.read_parquet('/work/project_maternity/data/activity', engine='pyarrow')

In [16]:
data_activity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25336286 entries, 0 to 25336285
Data columns (total 21 columns):
 #   Column             Dtype              
---  ------             -----              
 0   Module_Key         object             
 1   Date_Key           object             
 2   Time_Key           object             
 3   VideoChapter_Key   object             
 4   Userinfo_Key       object             
 5   Feature_Key        object             
 6   Drug_Key           object             
 7   Procedure_Key      object             
 8   ActionCard_Key     object             
 9   Chapter_Key        object             
 10  Language_Key       object             
 11  Lat                object             
 12  Lon                object             
 13  EventStart         datetime64[ns, UTC]
 14  EventEnd           datetime64[ns, UTC]
 15  SessionDuration    float64            
 16  SessId             object             
 17  AppId              object             
 18  

In [17]:
data_activity['Userinfo_Key']=pd.to_numeric(data_activity['Userinfo_Key'])

## Module_key

In [18]:
data_activity['Module_Key'].unique()

array(['473104', '-1', '473107', '603672', '473109', '473100', '473105',
       '631971', '473101', '473108', '473110', '473102', '595063',
       '649662', '473106', '473103', '786735'], dtype=object)

In [19]:
#just looking into module_Key and Userinfo_key
data_module = data_activity[['Userinfo_Key', 'Module_Key']]

In [20]:
data_module.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25336286 entries, 0 to 25336285
Data columns (total 2 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   Userinfo_Key  int64 
 1   Module_Key    object
dtypes: int64(1), object(1)
memory usage: 386.6+ MB


In [21]:
## Replacing all the -1 values with nan, as we suppose -1 represents the missing values. 
# its because when we see the most repetative module_key we don't get the -1 but get some values as the result. 
data_module.loc[data_module['Module_Key']== '-1', 'Module_Key'] = np.nan

In [22]:
data_module['Module_Key'].unique()

array(['473104', nan, '473107', '603672', '473109', '473100', '473105',
       '631971', '473101', '473108', '473110', '473102', '595063',
       '649662', '473106', '473103', '786735'], dtype=object)

In [23]:
# getting the most valuecounts model_key per userinfo_key
data_module = pd.DataFrame(data_module.groupby(['Userinfo_Key','Module_Key']).value_counts() 
   .sort_values(ascending=False) 
   .reset_index(name='count') 
   .drop_duplicates(subset='Userinfo_Key'))

In [24]:
data_module.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28199 entries, 0 to 144590
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Userinfo_Key  28199 non-null  int64 
 1   Module_Key    28199 non-null  object
 2   count         28199 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 881.2+ KB


In [25]:
data_module['Userinfo_Key'].isin(user_data['Userinfo_Key']).sum()
# seems like all the Userinfo_key in data_module is not inside user_data
# but lets merge the users who is inside user_data 

25358

In [26]:
# merging Module_Key variable from data_module to user_data based on Userinfo_Key
user_data = pd.merge(user_data, data_module[['Userinfo_Key', 'Module_Key']], on="Userinfo_Key", how="left")

In [27]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105914 entries, 0 to 105913
Data columns (total 21 columns):
 #   Column                            Non-Null Count   Dtype              
---  ------                            --------------   -----              
 0   Userinfo_Key                      105914 non-null  int64              
 1   Userinfo_BK                       105914 non-null  object             
 2   Country_Key                       105914 non-null  object             
 3   UserinfoAppId                     105914 non-null  object             
 4   RN                                105914 non-null  float64            
 5   OnlyUser                          105914 non-null  object             
 6   NumberOfUsers                     105914 non-null  object             
 7   UserinfoCountry                   105914 non-null  object             
 8   UserinfoState                     105914 non-null  object             
 9   UserinfoDistrict                  105914 non-nul

## looking for the average time used per user
- In this sectioin, we look into SessionDuration and EventStart, and find the average time per month per user. 

In [28]:
data_activity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25336286 entries, 0 to 25336285
Data columns (total 21 columns):
 #   Column             Dtype              
---  ------             -----              
 0   Module_Key         object             
 1   Date_Key           object             
 2   Time_Key           object             
 3   VideoChapter_Key   object             
 4   Userinfo_Key       int64              
 5   Feature_Key        object             
 6   Drug_Key           object             
 7   Procedure_Key      object             
 8   ActionCard_Key     object             
 9   Chapter_Key        object             
 10  Language_Key       object             
 11  Lat                object             
 12  Lon                object             
 13  EventStart         datetime64[ns, UTC]
 14  EventEnd           datetime64[ns, UTC]
 15  SessionDuration    float64            
 16  SessId             object             
 17  AppId              object             
 18  

In [29]:
data_time= data_activity[['SessionDuration', 'EventStart', 'Userinfo_Key']]
#focusing in these vaiables

In [30]:
data_time.SessionDuration = data_time.SessionDuration.fillna(0)
#filling missing values by 0 as we suppose missing session duration implies users not opening that session 

In [31]:
# Separating year and month from EventStart variable 
data_time["year"] = data_time["EventStart"].dt.year
data_time["month"] = data_time["EventStart"].dt.month


In [32]:
data_time= data_time.loc[:, ['SessionDuration', 'Userinfo_Key', 'year', 'month']]

In [33]:
data_time.month.unique()

array([ 2,  1,  3, 11, 10, 12,  9,  8])

In [34]:
data_time

,SessionDuration,Userinfo_Key,year,month
0,0.0,657254,2022,2
1,4.0,774423,2022,2
2,91.0,774423,2022,2
3,7.0,774423,2022,2
4,5.0,774423,2022,2
...,...,...,...,...
25336281,189.0,775997,2021,12
25336282,22.0,775997,2021,12
25336283,42.0,775997,2021,12
25336284,73.0,775997,2021,12


In [35]:
# mapping the month 
data_time['Month_Name'] = data_time['month'].map({1:'Jan', 2:'Feb', 3:'Mar', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'})

In [36]:
#Using pivot table to get mean duration that user used per month 
Avg_data_time = pd.pivot_table(data_time, index='Userinfo_Key', values='SessionDuration', columns='Month_Name', aggfunc='mean', margins=True)

In [37]:
Avg_data_time= Avg_data_time.reset_index()

In [38]:
Avg_data_time = Avg_data_time.replace('Nan', np.nan)

In [39]:
Avg_data_time

Month_Name,Userinfo_Key,Aug,Dec,Feb,Jan,Mar,Nov,Oct,Sep,All
0,474365,NaN,NaN,NaN,NaN,NaN,NaN,7.00000,NaN,7.000000
1,474442,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000
2,474687,NaN,NaN,NaN,NaN,NaN,NaN,4.00000,NaN,4.000000
3,474737,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.416667,3.416667
4,474986,NaN,NaN,0.333333,NaN,NaN,NaN,37.00000,2.629630,4.562500
...,...,...,...,...,...,...,...,...,...,...
39643,948550,NaN,NaN,NaN,NaN,1.666667,NaN,NaN,4.775000,4.369565
39644,948744,NaN,NaN,NaN,NaN,34.116279,NaN,NaN,NaN,34.116279
39645,948886,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,NaN,1.000000
39646,949140,NaN,NaN,NaN,NaN,NaN,NaN,2.00000,NaN,2.000000


In [40]:
Avg_data_time = Avg_data_time.fillna(0)

In [41]:
Avg_data_time

Month_Name,Userinfo_Key,Aug,Dec,Feb,Jan,Mar,Nov,Oct,Sep,All
0,474365,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,7.00000,0.000000,7.000000
1,474442,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
2,474687,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,4.00000,0.000000,4.000000
3,474737,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,3.416667,3.416667
4,474986,0.000000,0.00000,0.333333,0.000000,0.000000,0.000000,37.00000,2.629630,4.562500
...,...,...,...,...,...,...,...,...,...,...
39643,948550,0.000000,0.00000,0.000000,0.000000,1.666667,0.000000,0.00000,4.775000,4.369565
39644,948744,0.000000,0.00000,0.000000,0.000000,34.116279,0.000000,0.00000,0.000000,34.116279
39645,948886,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000,1.000000
39646,949140,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.00000,0.000000,2.000000


In [42]:
Avg_data_time['Userinfo_Key']= Avg_data_time['Userinfo_Key'].replace('All', np.nan)

In [43]:
Avg_data_time.dropna(inplace= True)

In [44]:
Avg_data_time

Month_Name,Userinfo_Key,Aug,Dec,Feb,Jan,Mar,Nov,Oct,Sep,All
0,474365.0,0.0,0.0,0.000000,0.0,0.000000,0.0,7.0,0.000000,7.000000
1,474442.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,474687.0,0.0,0.0,0.000000,0.0,0.000000,0.0,4.0,0.000000,4.000000
3,474737.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,3.416667,3.416667
4,474986.0,0.0,0.0,0.333333,0.0,0.000000,0.0,37.0,2.629630,4.562500
...,...,...,...,...,...,...,...,...,...,...
39642,948514.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
39643,948550.0,0.0,0.0,0.000000,0.0,1.666667,0.0,0.0,4.775000,4.369565
39644,948744.0,0.0,0.0,0.000000,0.0,34.116279,0.0,0.0,0.000000,34.116279
39645,948886.0,0.0,0.0,0.000000,0.0,0.000000,0.0,1.0,0.000000,1.000000


In [45]:
Avg_data_time = Avg_data_time.rename(columns={'All': 'Time'})

In [46]:
# merging time repesenting variable (ALL) from Avg_data_time to user_data based on Userinfo_Key
user_data = pd.merge(user_data, Avg_data_time[['Userinfo_Key', 'Time']], on="Userinfo_Key", how="left")

In [47]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105914 entries, 0 to 105913
Data columns (total 22 columns):
 #   Column                            Non-Null Count   Dtype              
---  ------                            --------------   -----              
 0   Userinfo_Key                      105914 non-null  int64              
 1   Userinfo_BK                       105914 non-null  object             
 2   Country_Key                       105914 non-null  object             
 3   UserinfoAppId                     105914 non-null  object             
 4   RN                                105914 non-null  float64            
 5   OnlyUser                          105914 non-null  object             
 6   NumberOfUsers                     105914 non-null  object             
 7   UserinfoCountry                   105914 non-null  object             
 8   UserinfoState                     105914 non-null  object             
 9   UserinfoDistrict                  105914 non-nul

## looking into mylearning dataset

In [48]:
my_learning = pd.read_parquet('/work/project_maternity/data/mylearning/data_mylearning.parquet')

In [49]:
my_learning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462262 entries, 0 to 1462261
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   Module_Key      1462262 non-null  object 
 1   Date_Key        1462180 non-null  object 
 2   Time_Key        1462262 non-null  float64
 3   Userinfo_Key    1462262 non-null  object 
 4   SessId          1462262 non-null  object 
 5   AppId           1462262 non-null  object 
 6   KlpResultLevel  1462262 non-null  float64
 7   KlpResultStars  1462138 non-null  float64
 8   KlpResultScore  145372 non-null   float64
dtypes: float64(4), object(5)
memory usage: 100.4+ MB


In [50]:
#we wanted to add some sort of achievement scores of the users and we found KlpResultScore and KlpResultLevel from which we can see acheivement
print(my_learning['KlpResultScore'].isna().sum())
print(my_learning['KlpResultLevel'].isna().sum())
#since the missing values of KlpResultLevel is 0, using KlpResultLevel as the acheivement for users

1316890
0


In [51]:
my_learning= my_learning[['Userinfo_Key', 'KlpResultLevel']]

In [52]:
my_learning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462262 entries, 0 to 1462261
Data columns (total 2 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   Userinfo_Key    1462262 non-null  object 
 1   KlpResultLevel  1462262 non-null  float64
dtypes: float64(1), object(1)
memory usage: 22.3+ MB


In [53]:
my_learning['Userinfo_Key'] = pd.to_numeric(my_learning['Userinfo_Key'])

In [54]:
print(len(my_learning['Userinfo_Key'].unique()))
print(len(my_learning['KlpResultLevel'].unique()))

14285
4


In [55]:
# getting mean of KlpResultLevel per each user
my_learning_level =pd.DataFrame(my_learning.groupby(['Userinfo_Key'])['KlpResultLevel'].mean())


In [56]:
my_learning_level= my_learning_level.reset_index()

In [57]:
my_learning_level.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14285 entries, 0 to 14284
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Userinfo_Key    14285 non-null  int64  
 1   KlpResultLevel  14285 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 223.3 KB


In [58]:
my_learning_level['Userinfo_Key'].isin(user_data['Userinfo_Key']).sum()


14285

In [59]:
user_data = pd.merge(user_data, my_learning_level[['Userinfo_Key', 'KlpResultLevel']], on="Userinfo_Key", how="left")

# Data Cleaning for further analysis

In [60]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105914 entries, 0 to 105913
Data columns (total 23 columns):
 #   Column                            Non-Null Count   Dtype              
---  ------                            --------------   -----              
 0   Userinfo_Key                      105914 non-null  int64              
 1   Userinfo_BK                       105914 non-null  object             
 2   Country_Key                       105914 non-null  object             
 3   UserinfoAppId                     105914 non-null  object             
 4   RN                                105914 non-null  float64            
 5   OnlyUser                          105914 non-null  object             
 6   NumberOfUsers                     105914 non-null  object             
 7   UserinfoCountry                   105914 non-null  object             
 8   UserinfoState                     105914 non-null  object             
 9   UserinfoDistrict                  105914 non-nul

In [61]:
# filling missing values by 0 
user_data['KlpResultLevel'][np.isnan(user_data['KlpResultLevel'])] = 0
user_data['Time'][np.isnan(user_data['Time'])] = 0

In [86]:
# filling missing values by -1,as in activity dataset all the missing values were filled with -1. 
user_data.Module_Key = user_data.Module_Key.fillna('-1')


In [87]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91009 entries, 140 to 105913
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   RN                    91009 non-null  float64
 1   UserinfoCountry       91009 non-null  int64  
 2   HealthcareWorker      91009 non-null  int64  
 3   Profession            91009 non-null  int64  
 4   Experiance            91009 non-null  int64  
 5   WorkPlace             91009 non-null  int64  
 6   SafeDeliveryApp       91009 non-null  int64  
 7   UserinfoIsMyLearning  91009 non-null  float64
 8   UserinfoUserIsActive  91009 non-null  float64
 9   Module_Key            91009 non-null  object 
 10  Time                  91009 non-null  float64
 11  KlpResultLevel        91009 non-null  float64
dtypes: float64(5), int64(6), object(1)
memory usage: 9.0+ MB


In [64]:
# Replacing all the Unknown to nan
user_data= user_data.replace('Unknown', np.nan)

In [65]:
# looking into missing values of each variable
for i in user_data.columns:
    n_miss = user_data[i].isna().sum()
    perc = n_miss / user_data.shape[0] * 100
    print(f'> {i}, Missing: {n_miss} ({perc:.2f} %)')

> Userinfo_Key, Missing: 0 (0.00 %)
> Userinfo_BK, Missing: 0 (0.00 %)
> Country_Key, Missing: 0 (0.00 %)
> UserinfoAppId, Missing: 0 (0.00 %)
> RN, Missing: 0 (0.00 %)
> OnlyUser, Missing: 9090 (8.58 %)
> NumberOfUsers, Missing: 78053 (73.69 %)
> UserinfoCountry, Missing: 0 (0.00 %)
> UserinfoState, Missing: 11924 (11.26 %)
> UserinfoDistrict, Missing: 23618 (22.30 %)
> HealthcareWorker, Missing: 783 (0.74 %)
> Profession, Missing: 13370 (12.62 %)
> Experiance, Missing: 13640 (12.88 %)
> WorkPlace, Missing: 5310 (5.01 %)
> SafeDeliveryApp, Missing: 14351 (13.55 %)
> UserinfoNumberOfDelivery, Missing: 50329 (47.52 %)
> UserinfoIsMyLearning, Missing: 0 (0.00 %)
> UserinfoUserIsActive, Missing: 0 (0.00 %)
> UserInfoFirstMyLearningEventDate, Missing: 23092 (21.80 %)
> UserInfoFirstUseDate, Missing: 0 (0.00 %)
> Module_Key, Missing: 0 (0.00 %)
> Time, Missing: 0 (0.00 %)
> KlpResultLevel, Missing: 0 (0.00 %)


In [66]:
user_data[['Country_Key', 'UserinfoCountry']]
# India = 487380
# Ethoipia 487366

,Country_Key,UserinfoCountry
0,487380,India
1,487366,Ethiopia
2,487380,India
3,487380,India
4,487380,India
...,...,...
105909,487366,Ethiopia
105910,487380,India
105911,487380,India
105912,487380,India


In [67]:
# dropping some vaiables which doesn't seems important for imputation
# - UserinfoAppId, Userinfo_Key, Userinfo_BK are key with all unique values
# - UserInfoFirstMyLearningEventDate, UserInfoFirstUseDate are data and time variable which don’t make sense to impute based on.
# - NumberOfUsers, UserinfoNumberOfDelivery:both the variables has a lot  missing data, which wasn’t enough preexisting data to go off of
# - Country_Key : Country_key and UserinfoCountry provides same information(about the country), decided to move on with UserinfoCountry and drop Country_Key
# - UserinfoState, UserinfoDistrict: have restrictions like a district needs to be inside a particular state, probably made more sense to leave them out of imputation and decided to remove the two features. 
user_data.drop(['UserInfoFirstMyLearningEventDate', 'UserInfoFirstUseDate', 'UserinfoAppId','Userinfo_Key', 'NumberOfUsers', 'Userinfo_BK','UserinfoNumberOfDelivery', 'Country_Key', 'OnlyUser', 'UserinfoState', 'UserinfoDistrict', ], axis=1, inplace=True)

In [68]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105914 entries, 0 to 105913
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   RN                    105914 non-null  float64
 1   UserinfoCountry       105914 non-null  object 
 2   HealthcareWorker      105131 non-null  object 
 3   Profession            92544 non-null   object 
 4   Experiance            92274 non-null   object 
 5   WorkPlace             100604 non-null  object 
 6   SafeDeliveryApp       91563 non-null   object 
 7   UserinfoIsMyLearning  105914 non-null  float64
 8   UserinfoUserIsActive  105914 non-null  float64
 9   Module_Key            105914 non-null  object 
 10  Time                  105914 non-null  float64
 11  KlpResultLevel        105914 non-null  float64
dtypes: float64(5), object(7)
memory usage: 10.5+ MB


In [69]:
user_data.dropna(inplace = True)

In [70]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91009 entries, 140 to 105913
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   RN                    91009 non-null  float64
 1   UserinfoCountry       91009 non-null  object 
 2   HealthcareWorker      91009 non-null  object 
 3   Profession            91009 non-null  object 
 4   Experiance            91009 non-null  object 
 5   WorkPlace             91009 non-null  object 
 6   SafeDeliveryApp       91009 non-null  object 
 7   UserinfoIsMyLearning  91009 non-null  float64
 8   UserinfoUserIsActive  91009 non-null  float64
 9   Module_Key            91009 non-null  object 
 10  Time                  91009 non-null  float64
 11  KlpResultLevel        91009 non-null  float64
dtypes: float64(5), object(7)
memory usage: 9.0+ MB


In [90]:
user_data['Module_Key'] = pd.to_numeric(user_data['Module_Key'])

In [91]:
label = LabelEncoder()

In [93]:
# changing string labels into numeric
user_data.Profession= label.fit_transform(user_data.Profession)
user_data.Experiance= label.fit_transform(user_data.Experiance)
user_data.WorkPlace= label.fit_transform(user_data.WorkPlace)
user_data.SafeDeliveryApp= label.fit_transform(user_data.SafeDeliveryApp)
user_data.HealthcareWorker = label.fit_transform(user_data.HealthcareWorker)
user_data.UserinfoCountry = label.fit_transform(user_data.UserinfoCountry)
user_data.Module_Key = label.fit_transform(user_data.Module_Key)

In [95]:
user_data.to_csv('clean_data.csv', index=False)